In [1]:
import tensorflow as tf
from tensorflow import keras
import math
import numpy as np

In [ ]:
saved_model_path = "" #insert the path where you saved the model
model = keras.models.load_model(saved_model_path)
model.pop()
model.summary()

In [ ]:
gallery_dir = gallery_path = '' #path to gallery dataset
query_dir = query_path = '' #path to query dataset

img_size = # @param [224, 300] insert the image size of the network you want to evaluate 

datagen_kwargs_nosplit = dict(rescale=1./255)
dataflow_kwargs_nosplit = dict(target_size=(img_size, img_size),
                       batch_size=1,
                       interpolation="bilinear")

gallery_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    **datagen_kwargs_nosplit)
gallery_generator = gallery_datagen.flow_from_directory(
    gallery_dir, subset=None, shuffle=False, **dataflow_kwargs_nosplit)

query_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    **datagen_kwargs_nosplit)
query_generator = query_datagen.flow_from_directory(
    query_dir, subset=None, shuffle=False, **dataflow_kwargs_nosplit)

In [ ]:
def get_class_string_from_index(index, generator):
   for class_string, class_index in generator.class_indices.items():
      if class_index == index:
         return class_string

def nextImage(generator, idx):  
  x, y = next(generator)
  images = []
  true_index = []
  paths_to_images = []
  for i in range(len(x)):
    images.append(x[i, :, :, :])
    true_idx = np.argmax(y[i])
    true_index.append(get_class_string_from_index(true_idx, generator))
    paths_to_images.append(generator.filepaths[idx])
    idx+=1
  return images, true_index, paths_to_images, idx

gallery_prob = []
gallery_labels = []
gallery_paths = []
n=len(gallery_generator.filepaths)
idx=0
for i in range(len(gallery_generator.filepaths)):
    image, true_index, paths_to_images, idx = nextImage(gallery_generator, idx)
    gallery_paths = [*gallery_paths, *paths_to_images]
    gallery_labels = [*gallery_labels, *true_index]
    probs_list = []
    for j in range(len(image)):
      probs_list.append(model.predict(np.expand_dims(image[j], axis=0)))
      
    gallery_prob = [*gallery_prob, *probs_list]
    print(f"loading gallery image batch {i+1}/{n}")



In [ ]:
query_prob = []
query_labels = []
query_paths = []
n=len(query_generator.filepaths)
idx=0
for i in range(len(query_generator.filepaths)):
    image, true_index, paths_to_images, idx = nextImage(query_generator, idx)
    query_paths = [*query_paths, *paths_to_images]
    query_labels = [*query_labels, *true_index]
    probs_list = []
    for j in range(len(image)):
      probs_list.append(model.predict(np.expand_dims(image[j], axis=0)))

    query_prob = [*query_prob, *probs_list]
    print(f"loading query image batch {i+1}/{n}")

In [ ]:
from scipy import spatial

top1 = 0
top5 = 0
top10 = 0
non_top = 0
for j in range (len(query_labels)):   
    distances = []
    np.set_printoptions(suppress=True)
    for i in range (len(gallery_prob)):
        d = 1-spatial.distance.cosine(query_prob[j], gallery_prob[i])
        distances.append([d, i])

    distances.sort(reverse=True)
    q_label = (query_labels[j])
    top_ten = distances[:10]
    isthere = False
    for k in range (len(top_ten)):
        if (gallery_labels[top_ten[k][1]]) == q_label:
            print(f"{gallery_labels[top_ten[0][1]]} = {q_label}")
            isthere = True
            if (k+1) == 1:
                top1+=1
                break
            elif (k+1) <=5:
                top5+=1
                break
            elif (k+1) <= 10:
                top10+=1
                break
    if not isthere:
        non_top +=1
        print(f"Query number {j} has no top-10 appearences")
        
            
print("top1: "+ str(top1))
print("top5: "+ str(top5))
print("top10: "+ str(top10))
print("Non top: " + str(non_top))
top1 = top1/len(query_prob)*100
top5 = top5/len(query_prob)*100 + top1
top10 = top10/len(query_prob)*100 + top5
print("top1: " + str(top1) + "%")
print("top5: " + str(top5)+ "%")
print("top10: " + str(top10)+ "%")

In [ ]:
from IPython import display

query_idx = 13
query_file = query_paths[query_idx]
query_class = query_labels[query_idx]

distances = []
np.set_printoptions(suppress=True)
for i in range (len(gallery_prob)):
    d = 1-spatial.distance.cosine(query_prob[query_idx], gallery_prob[i])
    distances.append([d, i])

distances.sort(reverse=True)
q_label = (query_labels[query_idx])
top_ten = distances[:10]

    
print(query_class)
display.display(display.Image(query_file))

import matplotlib.pylab as plt
import cv2

fig = plt.figure(figsize=(15, 15))

for i in range(len(top_ten)-1):
    fig.add_subplot(3, 3, (i+1))
    plt.imshow(cv2.imread(gallery_paths[top_ten[i][1]]))
    plt.axis('off')
    plt.title(gallery_labels[top_ten[i][1]]) 
    print(gallery_labels[top_ten[i][1]])
    print(gallery_paths[top_ten[i][1]])